In [4]:
import numpy as np
import sbi 

import lal as _lal
from pycbc.waveform import get_fd_waveform
from pycbc.psd import aLIGOZeroDetHighPower
from pycbc.filter import highpass

import matplotlib
import matplotlib.pyplot as plt

#### (chatGPT) The noise:

detectore noise is given by:

$$n(f) - \mathbb{C} N(0,0.5S_n(f))$$

where $S_n(f)$ is the one-sided power spectral density at frequency f.

In [36]:
"""
theta[0]: mass1 (solar masses)
theta[1]: mass2 (solar masses)
theta[2]: distance (Mpc)
"""

def simulator(theta, delta_f, f_lower, f_final):
    num_sim = theta.shape[0]
    num_freq = int(((f_final - f_lower) / delta_f) + 1)
    h_freq = np.zeros(shape = (num_sim, 4, num_freq))

    # Calculate the gravitational wave strains in frequency space num_sim times
    for i in range(num_sim):
        mass1 = theta[i,0]
        mass2 = theta[i,1]
        distance = theta[i,2]
        hp, hc = get_fd_waveform(approximant="TaylorF2",
                                 mass1=mass1,
                                 mass2=mass2,
                                 delta_f=delta_f,
                                 f_lower=f_lower,
                                 f_final=f_final,
                                 distance=distance,
                                 inclination=0.0,
                                 coa_phase=0.0)
        
        hp_freq = hp.to_frequencyseries()
        frequencies = hp_freq.sample_frequencies.numpy()
        hp_amplitudes = hp_freq.numpy()[frequencies >= f_lower]
        hp_amplitudes_real = np.real(hp_amplitudes)
        hp_amplitudes_imaginary = np.imag(hp_amplitudes)
        
        hc_freq = hc.to_frequencyseries()
        hc_amplitudes = hc_freq.numpy()[frequencies >= f_lower]
        hc_amplitudes_real = np.real(hc_amplitudes)
        hc_amplitudes_imaginary = np.imag(hc_amplitudes)
        
        h_freq[i,0,:] = hp_amplitudes_real
        h_freq[i,1,:] = hp_amplitudes_imaginary
        h_freq[i,2,:] = hc_amplitudes_real
        h_freq[i,3,:] = hc_amplitudes_imaginary
        
        hp_psd = aLIGOZeroDetHighPower(length = len(frequencies), delta_f = delta_f, low_freq_cutoff = f_lower).numpy()[frequencies >= f_lower]
        hp_scale = np.sqrt(0.5 * hp_psd * delta_f)
        hp_noise_real = np.random.normal(0, hp_scale)
        hp_noise_imaginary = np.random.normal(0, hp_scale)
        
        hc_psd = aLIGOZeroDetHighPower(length = len(frequencies), delta_f = delta_f, low_freq_cutoff = f_lower).numpy()[frequencies >= f_lower]
        hc_scale = np.sqrt(0.5 * hc_psd * delta_f)
        hc_noise_real = np.random.normal(0, hc_scale)
        hc_noise_imaginary = np.random.normal(0, hc_scale)
        
        h_freq[i,0,:] += hp_noise_real
        h_freq[i,1,:] += hp_noise_imaginary
        h_freq[i,2,:] += hc_noise_real
        h_freq[i,3,:] += hc_noise_imaginary

    return h_freq
    
    

In [32]:
mass1 = 1
mass2 = 2
distance = 500
delta_f = 1/16
f_lower = 20.0
f_final = 512.0

hp, hc = get_fd_waveform(approximant="TaylorF2",
                             mass1=mass1,
                             mass2=mass2,
                             delta_f=delta_f,
                             f_lower=f_lower,
                             f_final=f_final,
                             distance=distance,
                             inclination=0.0,
                             coa_phase=0.0)

ht = hp.to_timeseries()
times = ht.sample_times.numpy()
amplitude = ht.numpy()

plt.plot(times, amplitude)
plt.xlabel("Time (s)")
plt.ylabel("Strain h(t)")
plt.savefig("time series")
plt.close()

hp_freq = hp.to_frequencyseries()
frequencies = hp_freq.sample_frequencies.numpy()
hp_amplitudes = hp_freq.numpy()
hp_amplitudes_real = np.real(hp_amplitudes)
hp_amplitudes_imaginary = np.imag(hp_amplitudes)

hc_freq = hc.to_frequencyseries()
hc_amplitudes = hc_freq.numpy()
hc_amplitudes_real = np.real(hc_amplitudes)
hc_amplitudes_imaginary = np.imag(hc_amplitudes)

h_freq = np.zeros(shape = (4, len(frequencies)))
h_freq[0,:] = hp_amplitudes_real
h_freq[1,:] = hp_amplitudes_imaginary
h_freq[2,:] = hc_amplitudes_real
h_freq[3,:] = hc_amplitudes_imaginary

hp_psd = aLIGOZeroDetHighPower(length = len(frequencies), delta_f = delta_f, low_freq_cutoff = f_lower)
hp_scale = np.sqrt(0.5 * hp_psd.numpy() * delta_f)
hp_noise_real = np.random.normal(0, hp_scale)
hp_noise_imaginary = np.random.normal(0, hp_scale)

hc_psd = aLIGOZeroDetHighPower(length = len(frequencies), delta_f = delta_f, low_freq_cutoff = f_lower)
hc_scale = np.sqrt(0.5 * hc_psd.numpy() * delta_f)
hc_noise_real = np.random.normal(0, hc_scale)
hc_noise_imaginary = np.random.normal(0, hc_scale)

h_freq[0,:] += hp_noise_real
h_freq[1,:] += hp_noise_imaginary
h_freq[2,:] += hc_noise_real
h_freq[3,:] += hc_noise_imaginary

plt.plot(frequencies, h_freq[0,:])
plt.plot(frequencies, h_freq[1,:])
plt.xlabel('frequency (Hz)')
plt.ylabel('Strain amplitude h(f)')
plt.savefig("frequency series hp + noise")
plt.close()

plt.plot(frequencies, h_freq[2,:])
plt.plot(frequencies, h_freq[3,:])
plt.xlabel('frequency (Hz)')
plt.ylabel('Strain amplitude h(f)')
plt.savefig("frequency series hc + noise")
plt.close()

In [37]:
theta = np.array([[30,30,100],
                 [40,15,200],
                 [50,25,300],
                 [20,40,400],
                 [35,20,500],])
h_freq = simulator(theta, delta_f, f_lower, f_final)

In [38]:
h_freq[0,:,:]

array([[-3.45226834e-22, -3.62603016e-22, -3.39443943e-22, ...,
         7.15074721e-25,  1.35410652e-24,  7.33152430e-25],
       [ 1.09533872e-22, -2.48258459e-24, -1.10218060e-22, ...,
        -7.02355994e-24, -8.19829895e-24, -8.14050027e-24],
       [ 1.17572283e-22, -2.25719739e-24, -1.12382441e-22, ...,
        -9.75562033e-24, -7.37974689e-24, -8.14050027e-24],
       [ 3.46883991e-22,  3.69961106e-22,  3.43698316e-22, ...,
        -7.95158108e-25, -5.44850684e-25, -7.33152430e-25]])

In [34]:
len(hp_noise)

7873

In [30]:
aLIGOZeroDetHighPower(length = len(frequencies), delta_f = delta_f, low_freq_cutoff = f_lower).plot()
plt.savefig('noise')
plt.close()

In [17]:
frequencies = frequencies[frequencies >= f_lower]

In [18]:
len(frequencies)

7873

In [35]:
hp_noise

array([4.28646109e-46, 4.22982019e-46, 4.17414163e-46, ...,
       1.52467903e-47, 1.52477428e-47, 0.00000000e+00])